In [1]:
from kivy.app import App
from os.path import sep, expanduser, isdir, dirname

from kivy.uix.boxlayout import BoxLayout
from kivy.uix.treeview import TreeViewLabel, TreeView
try:
    from kivy.garden.filechooserthumbview import FileChooserThumbView as \
        IconView
except (ImportError, KeyError):
    from kivy.uix.filechooser import FileChooserIconView as IconView
from kivy.properties import (ObjectProperty, StringProperty, OptionProperty, ListProperty, BooleanProperty)
from kivy.lang import Builder
from kivy.utils import platform
from kivy.clock import Clock
from kivy.compat import PY2
import string
from os.path import sep, dirname, expanduser, isdir, join
from os import walk
from sys import getfilesystemencoding
from functools import partial

if platform == 'win':
    from ctypes import windll, create_unicode_buffer


__all__ = ('FileBrowser', )


def get_home_directory():
    if platform == 'win':
        user_path = expanduser('~')

        if not isdir(join(user_path, 'Desktop')):
            user_path = dirname(user_path)

    else:
        user_path = expanduser('~')

    if PY2:
        user_path = user_path.decode(getfilesystemencoding())

    return user_path


def get_drives():
    drives = []
    if platform == 'win':
        bitmask = windll.kernel32.GetLogicalDrives()
        GetVolumeInformationW = windll.kernel32.GetVolumeInformationW
        for letter in string.ascii_uppercase:
            if bitmask & 1:
                name = create_unicode_buffer(64)
                # get name of the drive
                drive = letter + u':'
                res = GetVolumeInformationW(drive + sep, name, 64, None,
                                            None, None, None, 0)
                if isdir(drive):
                    drives.append((drive, name.value))
            bitmask >>= 1
    elif platform == 'linux':
        drives.append((sep, sep))
        drives.append((expanduser(u'~'), '~/'))
        places = (sep + u'mnt', sep + u'media')
        for place in places:
            if isdir(place):
                for directory in next(walk(place))[1]:
                    drives.append((place + sep + directory, directory))
    elif platform == 'macosx' or platform == 'ios':
        drives.append((expanduser(u'~'), '~/'))
        vol = sep + u'Volume'
        if isdir(vol):
            for drive in next(walk(vol))[1]:
                drives.append((vol + sep + drive, drive))
    return drives


class FileBrowserIconView(IconView):
    pass


Builder.load_string('''
#:kivy 1.2.0
#:import metrics kivy.metrics
#:import abspath os.path.abspath
<TreeLabel>:
    on_touch_down:
        self.parent.browser.current_tab.content.path = self.path if\
        self.collide_point(*args[1].pos) and self.path else\
        self.parent.browser.current_tab.content.path
    on_is_open: self.is_open and self.parent.trigger_populate(self)
<FileBrowser>:
    orientation: 'vertical'
    spacing: 5
    padding: [6, 6, 6, 6]
    select_state: select_button.state
    cancel_state: cancel_button.state
    filename: file_text.text
    on_favorites: link_tree.reload_favs(self.favorites)
    BoxLayout:
        orientation: 'horizontal'
        spacing: 5
        Splitter:
            sizable_from: 'right'
            min_size: '153sp'
            size_hint: (.2, 1)
            id: splitter
            ScrollView:
                LinkTree:
                    id: link_tree
                    browser: tabbed_browser
                    size_hint_y: None
                    height: self.minimum_height
                    on_parent: self.fill_tree(root.favorites)
                    root_options: {'text': 'Locations', 'no_selection':True}
        BoxLayout:
            size_hint_x: .8
            orientation: 'vertical'
            Label:
                size_hint_y: None
                height: '22dp'
                text_size: self.size
                padding_x: '10dp'
                text: abspath(root.path)
                valign: 'middle'
            TabbedPanel:
                id: tabbed_browser
                do_default_tab: False
                TabbedPanelItem:
                    text: 'List View'
                    FileChooserListView:
                        id: list_view
                        path: root.path
                        filters: root.filters
                        filter_dirs: root.filter_dirs
                        show_hidden: root.show_hidden
                        multiselect: root.multiselect
                        dirselect: root.dirselect
                        rootpath: root.rootpath
                        on_submit: root.dispatch('on_submit')
                TabbedPanelItem:
                    text: 'Icon View'
                    FileBrowserIconView:
                        id: icon_view
                        path: root.path
                        filters: root.filters
                        filter_dirs: root.filter_dirs
                        show_hidden: root.show_hidden
                        multiselect: root.multiselect
                        dirselect: root.dirselect
                        rootpath: root.rootpath
                        on_submit: root.dispatch('on_submit')
    GridLayout:
        size_hint: (1, None)
        height: file_text.line_height * 4
        cols: 2
        rows: 2
        spacing: [5]
        TextInput:
            id: file_text
            text: (root.selection and (root._shorten_filenames(\
            root.selection) if root.multiselect else root.selection[0])) or ''
            hint_text: 'Filename'
            multiline: False
        Button:
            id: select_button
            size_hint_x: None
            width: metrics.dp(100)
            text: root.select_string
            on_release: root.dispatch('on_success')
        TextInput:
            id: filt_text
            hint_text: '*.*'
            on_text_validate:
                root.filters = self.text.split(',') if self.text else []
            multiline: False
            text: ','.join(\
[filt for filt in root.filters if isinstance(filt, str)])
        Button:
            id: cancel_button
            size_hint_x: None
            width: metrics.dp(100)
            text: root.cancel_string
            on_release: root.dispatch('on_canceled')
''')


class TreeLabel(TreeViewLabel):
    path = StringProperty('')
    '''Full path to the location this node points to.
    :class:`~kivy.properties.StringProperty`, defaults to ''
    '''


class LinkTree(TreeView):
    # link to the favorites section of link bar
    _favs = ObjectProperty(None)
    _computer_node = None

    def fill_tree(self, fav_list):
        user_path = get_home_directory()
        self._favs = self.add_node(TreeLabel(text='Favorites', is_open=True,
                                             no_selection=True))
        self.reload_favs(fav_list)

        libs = self.add_node(TreeLabel(text='Libraries', is_open=True,
                                       no_selection=True))
        places = ('Documents', 'Music', 'Pictures', 'Videos')
        for place in places:
            if isdir(join(user_path, place)):
                self.add_node(TreeLabel(text=place, path=join(user_path,
                                        place)), libs)
        self._computer_node = self.add_node(TreeLabel(
            text='Computer', is_open=True, no_selection=True))
        self._computer_node.bind(on_touch_down=self._drives_touch)
        self.reload_drives()

    def _drives_touch(self, obj, touch):
        if obj.collide_point(*touch.pos):
            self.reload_drives()

    def reload_drives(self):
        nodes = [(node, node.text + node.path)
                 for node in self._computer_node.nodes
                 if isinstance(node, TreeLabel)]
        sigs = [s[1] for s in nodes]
        nodes_new = []
        sig_new = []
        for path, name in get_drives():
            if platform == 'win':
                text = u'{}({})'.format((name + ' ') if name else '', path)
            else:
                text = name
            nodes_new.append((text, path))
            sig_new.append(text + path + sep)
        for node, sig in nodes:
            if sig not in sig_new:
                self.remove_node(node)
        for text, path in nodes_new:
            if text + path + sep not in sigs:
                self.add_node(TreeLabel(text=text, path=path + sep),
                              self._computer_node)

    def reload_favs(self, fav_list):
        user_path = get_home_directory()
        favs = self._favs
        remove = []
        for node in self.iterate_all_nodes(favs):
            if node != favs:
                remove.append(node)
        for node in remove:
            self.remove_node(node)
        places = ('Desktop', 'Downloads')
        for place in places:
            if isdir(join(user_path, place)):
                self.add_node(TreeLabel(text=place, path=join(user_path,
                                        place)), favs)
        for path, name in fav_list:
            if isdir(path):
                self.add_node(TreeLabel(text=name, path=path), favs)

    def trigger_populate(self, node):
        if not node.path or node.nodes:
            return
        parent = node.path
        _next = next(walk(parent))
        if _next:
            for path in _next[1]:
                self.add_node(TreeLabel(text=path, path=parent + sep + path),
                              node)


class FileBrowser(BoxLayout):

    __events__ = ('on_canceled', 'on_success', 'on_submit')

    select_state = OptionProperty('normal', options=('normal', 'down'))
  
    cancel_state = OptionProperty('normal', options=('normal', 'down'))
   
    select_string = StringProperty('Ok')
  
    cancel_string = StringProperty('Cancel')

    filename = StringProperty('')
   
    selection = ListProperty([])
    
    path = StringProperty(u'/')

    filters = ListProperty([])

    filter_dirs = BooleanProperty(False)

    show_hidden = BooleanProperty(False)

    multiselect = BooleanProperty(False)

    dirselect = BooleanProperty(False)

    rootpath = StringProperty(None, allownone=True)

    favorites = ListProperty([])

    def on_success(self):
        pass

    def on_canceled(self):
        pass

    def on_submit(self):
        pass

    def __init__(self, **kwargs):
        super(FileBrowser, self).__init__(**kwargs)
        Clock.schedule_once(self._post_init)

    def _post_init(self, *largs):
        self.ids.icon_view.bind(
            selection=partial(self._attr_callback, 'selection'),
            path=partial(self._attr_callback, 'path'),
            filters=partial(self._attr_callback, 'filters'),
            filter_dirs=partial(self._attr_callback, 'filter_dirs'),
            show_hidden=partial(self._attr_callback, 'show_hidden'),
            multiselect=partial(self._attr_callback, 'multiselect'),
            dirselect=partial(self._attr_callback, 'dirselect'),
            rootpath=partial(self._attr_callback, 'rootpath'))
        self.ids.list_view.bind(
            selection=partial(self._attr_callback, 'selection'),
            path=partial(self._attr_callback, 'path'),
            filters=partial(self._attr_callback, 'filters'),
            filter_dirs=partial(self._attr_callback, 'filter_dirs'),
            show_hidden=partial(self._attr_callback, 'show_hidden'),
            multiselect=partial(self._attr_callback, 'multiselect'),
            dirselect=partial(self._attr_callback, 'dirselect'),
            rootpath=partial(self._attr_callback, 'rootpath'))

    def _shorten_filenames(self, filenames):
        if not len(filenames):
            return ''
        elif len(filenames) == 1:
            return filenames[0]
        elif len(filenames) == 2:
            return filenames[0] + ', ' + filenames[1]
        else:
            return filenames[0] + ', _..._, ' + filenames[-1]

    def _attr_callback(self, attr, obj, value):
        setattr(self, attr, getattr(obj, attr))


if __name__ == '__main__':
    import os
    from kivy.app import App

    class TestApp(App):

        def build(self):
            user_path = os.path.join(get_home_directory(), 'Documents')
            browser = FileBrowser(select_string='Select',
                                  favorites=[(user_path, 'Documents')])
            browser.bind(on_success=self._fbrowser_success,
                         on_canceled=self._fbrowser_canceled,
                         on_submit=self._fbrowser_submit)
            return browser

        def _fbrowser_canceled(self, instance):
            print('cancelled, Close self.')

        def _fbrowser_success(self, instance):
            print(instance.selection)

        def _fbrowser_submit(self, instance):
            print(instance.selection)

    TestApp().run()

[INFO   ] [Logger      ] Record log in /home/swathi/.kivy/logs/kivy_20-01-22_8.txt
[INFO   ] [Kivy        ] v1.11.1
[INFO   ] [Kivy        ] Installed at "/home/swathi/anaconda3/lib/python3.7/site-packages/kivy/__init__.py"
[INFO   ] [Python      ] v3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]
[INFO   ] [Python      ] Interpreter at "/home/swathi/anaconda3/bin/python"
[INFO   ] [Factory     ] 184 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil, img_gif (img_ffpyplayer ignored)
[INFO   ] [Text        ] Provider: sdl2
[INFO   ] [Window      ] Provider: sdl2(['window_egl_rpi'] ignored)
[INFO   ] [GL          ] Using the "OpenGL" graphics system
[INFO   ] [GL          ] Backend used <sdl2>
[INFO   ] [GL          ] OpenGL version <b'3.0 Mesa 18.2.8'>
[INFO   ] [GL          ] OpenGL vendor <b'Intel Open Source Technology Center'>
[INFO   ] [GL          ] OpenGL renderer <b'Mesa DRI Intel(R) HD Graphics 520 (Skylake GT2) '>
[INFO   ] [GL          

cancelled, Close self.
cancelled, Close self.


[INFO   ] [Base        ] Leaving application in progress...
[INFO   ] [WindowSDL   ] exiting mainloop and closing.
